In [1]:
import os
from pathlib import Path

current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
results_dir.mkdir(parents=True, exist_ok=True)
#examples_txt_path = results_dir.joinpath('examples.txt')

In [2]:
# Downloading and parsing the initial text file
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt', 
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 600893


In [3]:
# Vectorizing sequences of characters
maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


In [4]:
# Single-layer LSTM model for next-character prediction
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [5]:
# Model compilation configuration
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [6]:
# Function to sample the next character given the model’s predictions
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [7]:
# Text-generation loop
import random
import sys

for epoch in range(1, 20):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
            
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)
            
        if epoch <= 20 and temperature <= 1.2:
            example_txt = 'epoch' + str(epoch) + 'temperature' + str(temperature) + '.txt'
            example_txt_path = results_dir.joinpath(example_txt)
            f = open(example_txt_path, "w")
            f.write(generated_text)
            f.close()
            epoch += 1
            
model.save('model.h5')

epoch 1
1565/1565 [==============================] - 193s 123ms/step - loss: 1.9450
--- Generating with seed: "verything else, and called out
constantly to his pupils: "be"
------ temperature: 0.2
verything else, and called out
constantly to his pupils: "be and the same for the same the same the same and the same and the same and the surthes and the such as the same the same and the same in the same the simple of the part of the same the our insting the same the and many and the same the surches and the such and the same the instinct of the same in the same the such and the same for the manistentary that is the same the same the may part of the same------ temperature: 0.5
istentary that is the same the same the may part of the same to the germans of the fare and live we hard only the wist its im even may be on the antes the seen and resting is a conscience of his in the cause of the way far a fally that the something the instincts of many in the still a desind
their manity of the conte

is indepedues there
of knowards of non be most desirian of, in their something distily deed
are attain,
and
"him were thinkernates was
lend, as ther.

2me thereof
"shpaciciatingings, the influencegly,
werehart. hen be himself !  the hololy withepoch 5
1565/1565 [==============================] - 196s 125ms/step - loss: 1.4475
--- Generating with seed: " in philosophy, whatever solemn, whatever conclusive
and dec"
------ temperature: 0.2
 in philosophy, whatever solemn, whatever conclusive
and decise of the same to the religions of the spirit and something the sent to the self something when the sentiment to the seems the sense of the subject of the most and such a soul--and and something the sense of the sent to the sent to the sent to the same to be such a most distingure of the sent to the sent to the same and seems the sent to the serious and such a soul and the second of the suffer th------ temperature: 0.5
 the serious and such a soul and the second of the suffer the some one of t

es implice of natureed foundation among swmans. the modes in piefles of one or
hi------ temperature: 1.2
d foundation among swmans. the modes in piefles of one or
his ideate!"

1ed dus he no, for a joy.; thinks it is
sigted as and mecional beginings.=-anderhing thelingsand weak even to
mirather will instance of greatness. there ; is and series feelingwing--grotsrang. bed will pnesssed conscroacted; -i miawds of
injude my mood sottry-thinkselmaited.. but lines si. you now.

 in. st pall. he may begined to be .oleanthist, problem of concluping to often"-in the epoch 9
1565/1565 [==============================] - 211s 135ms/step - loss: 1.3848
--- Generating with seed: "ove for himself. if, indeed, his egoism really suffers more,"
------ temperature: 0.2
ove for himself. if, indeed, his egoism really suffers more, the stronger of the problem of the sense of the sense of the sense of the sense of the problem of the problem of the sense of the sense of the sense of the experience, and and t

tfo--"literly doubt
the existen, and the right in every perhapst germaneition: it are mer, sours, the miskne, there is more ;inter affece," about it is
accornation, there is unaquently from is in manifests that is there are last
hows to age and world out of and he
skebo their
good will, he becamority, an its world there educated.--possi------ temperature: 1.2
ood will, he becamority, an its world there educated.--possibilie stand to much be as thtom,
safional
notacr--modesticitimitely a godis ging.ive in all form of unconfracrismice,
has
time deteriours. un "and somited peofie, "a'sual is picts exacl
extencixment:-her efiednation
and
time long piod healt accivution schopenhauer. (who namely oneself, werar,ly, and living even will own
covodrary ase lukned;--choosed, raping to le meanse, but not animal truel. eveepoch 13
1565/1565 [==============================] - 198s 127ms/step - loss: 1.3520
--- Generating with seed: " are enthusiastic about the good, true, and beautiful,
and l"
----

ecisely of the existence of the belief of the sense of the fear the present than with the man is for the heart of the precisely of the power. namely in the free spirit and should be still phenost the word of the helpy of the sense of the morality of the servic and the are of the element of the suffering in man for the perhaps the world of the world of the man is present subverty it is a has scortions of the philosophers man of the significance and resulted------ temperature: 1.0
ons of the philosophers man of the significance and resulted be strene: all
condition, one is reasuquitians, fall;
what i canditadking which niy--an already over-eby sert, the philosophers, too man who mean for the icmases who suped, this betust the degree
of a bothouraguinration, what say, all laborisfuin of not for the power's intercould and loftiffius in the what go intellect am--yon emotione artists, "possible is sait; love
sopire? the spirit. attersit------ temperature: 1.2
rtists, "possible is sait; love
